In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from numpy.random import choice
from pandas import DataFrame
from sklearn.pipeline import Pipeline

C:\Users\chino\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [3]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,37,23,20,0,3,27,9,14,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,37,21,22,4,3,31,11,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,24,24,38,2,3,30,9,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [13]:
train['ID'].values

array([   0,    6,    7, ..., 8412, 8415, 8417], dtype=int64)

In [4]:
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,X242,X243,X244,X245,X246,X247,X248,X249,X250,X251,X252,X253,X254,X255,X256,X257,X258,X259,X260,X261,X262,X263,X264,X265,X266,X267,X268,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X279,X280,X281,X282,X283,X284,X285,X286,X287,X288,X289,X290,X291,X292,X293,X294,X295,X296,X297,X298,X299,X300,X301,X302,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320,X321,X322,X323,X324,X325,X326,X327,X328,X329,X330,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,24,23,38,5,3,26,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,46,3,9,0,3,9,6,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,24,23,19,5,3,0,9,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,

In [5]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [6]:
X

array([[   0,   37,   23, ...,    0,    0,    0],
       [   6,   37,   21, ...,    0,    0,    0],
       [   7,   24,   24, ...,    0,    0,    0],
       ..., 
       [8412,   10,   23, ...,    0,    0,    0],
       [8415,   11,   19, ...,    0,    0,    0],
       [8417,   52,   19, ...,    0,    0,    0]], dtype=int64)

In [7]:
X_boot=[]
y_boot=[]
for i in range(0,100):
    X_boot.append(resample(X, n_samples=300, random_state=i))
    y_boot.append(resample(y, n_samples=300, random_state=i))

In [8]:
X_boot=np.array(X_boot)
y_boot=np.array(y_boot)

In [9]:
X_boot

array([[[5457,    0,   23, ...,    0,    0,    0],
        [5219,   49,   20, ...,    0,    0,    0],
        [3313,   46,    3, ...,    0,    0,    0],
        ..., 
        [1128,   49,   20, ...,    0,    0,    0],
        [1576,   41,   20, ...,    0,    0,    0],
        [6074,   36,   11, ...,    0,    0,    0]],

       [[ 474,   50,   10, ...,    0,    0,    0],
        [7980,   49,   20, ...,    0,    0,    0],
        [1806,    0,   23, ...,    0,    0,    0],
        ..., 
        [1810,   52,    1, ...,    0,    0,    0],
        [2029,   41,   13, ...,    0,    0,    0],
        [3624,   30,    1, ...,    0,    0,    0]],

       [[5160,   32,    4, ...,    0,    0,    0],
        [7228,   23,   19, ...,    0,    0,    0],
        [5037,   36,    1, ...,    0,    0,    0],
        ..., 
        [6136,   36,    1, ...,    0,    0,    0],
        [7332,   41,   23, ...,    0,    0,    0],
        [3785,    9,   13, ...,    0,    0,    0]],

       ..., 
       [[4941,   51, 

In [ ]:
np.where(X_boot )

In [10]:
forest=RandomForestRegressor(n_estimators=500, random_state=0, max_depth=5)

In [11]:
kf=KFold(n_splits=5, shuffle=True, random_state=0)
for i in range(0,100):
    kf.get_n_splits(X_boot[i])

In [12]:
#X_train=[]
#X_test=[]
#y_train=[]
#y_test=[]
forest_level1_pred=[]
for i in range(0,100):
    for train_index, test_index in kf.split(X_boot[i]):
        X_train, y_train=X_boot[i][train_index], y_boot[i][train_index]
        X_valid, y_valid=X_boot[i][test_index], y_boot[i][test_index]
        
    forest.fit(X_train, y_train)
    forest_level1_pred.append(forest.predict(X_valid))
    
        
        #X_train.append(X_boot[i][train_index]) 
        #X_test.append(X_boot[i][test_index])
        #y_train.append(y_boot[i][train_index])
        #y_test.append(y_boot[i][test_index])

In [9]:
forest_level1_pred

[array([ 112.49831398,  104.6114791 ,  111.26861211,   93.88780523,
          99.53691615,  112.60647447,  110.37983613,   76.31154505,
          95.3355334 ,   79.58565364,   92.36555501,   91.37239389,
         109.66478189,   91.57617695,   91.99567356,  114.39350361,
          94.43127659,  110.73309329,   94.25423859,  111.1087991 ,
          93.01891959,   93.54198249,   93.59266287,  113.57340183,
          92.75389477,  103.32345075,   97.30250027,   92.56425562,
         111.84052471,   94.00495027,  106.19620463,  110.51901545,
          91.99567356,  112.07907933,   98.21589556,   97.80219732,
         101.20889989,   76.72963472,  111.23927597,   92.4571769 ,
         113.43216192,   93.63913271,  104.27961651,   94.10144834,
          95.60652428,  120.99085264,  113.11721257,  100.72758917,
         113.89245242,  100.5845354 ,   93.05234186,   97.16506253,
          93.15824397,   92.49620195,   76.73151253,   93.3666781 ,
         100.55933011,   97.78843933,  111.19063

In [24]:
xgb_params={
    'gamma': 1.0,
    'eta': 0.01,
    'max_depth': 3,
    'subsample':1.0,
    'objective' : 'reg:linear',
    'eval_metric': 'rmse',
    'base_score' : y_mean,
    'silent': 1,
    'lambda': 1,
    'alpha' : 0,
    'n_trees': 500
   }

xgb_level1_pred=[]
for i in range(0,100):
    for train_index, test_index in kf.split(X_boot[i]):
        X_train, y_train=X_boot[i][train_index], y_boot[i][train_index]
        X_valid, y_valid=X_boot[i][test_index], y_boot[i][test_index]
        
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid=xgb.DMatrix(X_valid, y_valid)

# xgboost, cross-validation
    cv_result = xgb.cv(xgb_params, 
                       dtrain, 
                       num_boost_round=500, # increase to have better results (~700)
                       early_stopping_rounds=50,
                       verbose_eval=50, 
                       show_stdv=False
                      )

    num_boost_rounds = len(cv_result)
    print(num_boost_rounds)

# train model
    model = xgb.train(dict(xgb_params, silent=0, n_jobs=3), dtrain, num_boost_round=num_boost_rounds)
    xgb_level1_pred.append(model.predict(dvalid))

[0]	train-rmse:12.3892	test-rmse:12.388
[50]	train-rmse:9.48262	test-rmse:9.82799
[100]	train-rmse:7.89734	test-rmse:8.74255
[150]	train-rmse:6.9208	test-rmse:8.32565
[200]	train-rmse:6.27938	test-rmse:8.20475
[250]	train-rmse:5.83108	test-rmse:8.21003
217
[0]	train-rmse:12.0493	test-rmse:12.0406
[50]	train-rmse:9.60216	test-rmse:10.2314
[100]	train-rmse:8.22508	test-rmse:9.56605
[150]	train-rmse:7.37985	test-rmse:9.31622
[200]	train-rmse:6.71745	test-rmse:9.18733
[250]	train-rmse:6.21375	test-rmse:9.07549
[300]	train-rmse:5.82082	test-rmse:9.02916
[350]	train-rmse:5.50397	test-rmse:8.95363
[400]	train-rmse:5.2452	test-rmse:8.91694
[450]	train-rmse:5.01676	test-rmse:8.89547
[499]	train-rmse:4.82573	test-rmse:8.88723
500
[0]	train-rmse:12.1109	test-rmse:12.0983
[50]	train-rmse:9.30614	test-rmse:9.56967
[100]	train-rmse:7.79485	test-rmse:8.53366
[150]	train-rmse:6.88902	test-rmse:8.12975
[200]	train-rmse:6.27016	test-rmse:7.98334
[250]	train-rmse:5.83894	test-rmse:7.96149
237
[0]	train-r

[50]	train-rmse:10.1834	test-rmse:10.519
[100]	train-rmse:8.63654	test-rmse:9.55285
[150]	train-rmse:7.69703	test-rmse:9.30708
[200]	train-rmse:7.05707	test-rmse:9.23131
[250]	train-rmse:6.60129	test-rmse:9.24668
218
[0]	train-rmse:12.9823	test-rmse:12.9644
[50]	train-rmse:10.0498	test-rmse:10.3745
[100]	train-rmse:8.30822	test-rmse:9.27091
[150]	train-rmse:7.21968	test-rmse:8.89803
[200]	train-rmse:6.51878	test-rmse:8.73208
[250]	train-rmse:5.99556	test-rmse:8.68376
245
[0]	train-rmse:12.1464	test-rmse:12.1516
[50]	train-rmse:9.50868	test-rmse:10.0752
[100]	train-rmse:7.90992	test-rmse:9.19693
[150]	train-rmse:6.87229	test-rmse:8.83237
[200]	train-rmse:6.18605	test-rmse:8.63173
[250]	train-rmse:5.7127	test-rmse:8.47836
[300]	train-rmse:5.32779	test-rmse:8.39355
[350]	train-rmse:5.0312	test-rmse:8.34392
[400]	train-rmse:4.79924	test-rmse:8.29539
[450]	train-rmse:4.59003	test-rmse:8.26897
[499]	train-rmse:4.40074	test-rmse:8.2428
500
[0]	train-rmse:11.6675	test-rmse:11.6699
[50]	train-r

[100]	train-rmse:7.33095	test-rmse:8.27888
[150]	train-rmse:6.60415	test-rmse:7.9651
[200]	train-rmse:6.04917	test-rmse:7.89502
189
[0]	train-rmse:13.0652	test-rmse:13.058
[50]	train-rmse:10.4525	test-rmse:10.7921
[100]	train-rmse:8.90467	test-rmse:9.62888
[150]	train-rmse:8.03075	test-rmse:9.02737
[200]	train-rmse:7.42961	test-rmse:8.69652
[250]	train-rmse:6.87105	test-rmse:8.48339
[300]	train-rmse:6.3684	test-rmse:8.38973
[350]	train-rmse:5.99097	test-rmse:8.29674
[400]	train-rmse:5.68437	test-rmse:8.28135
385
[0]	train-rmse:11.9809	test-rmse:11.8731
[50]	train-rmse:9.51806	test-rmse:9.79892
[100]	train-rmse:8.16022	test-rmse:9.02702
[150]	train-rmse:7.34908	test-rmse:8.71359
[200]	train-rmse:6.75725	test-rmse:8.60666
192
[0]	train-rmse:12.3933	test-rmse:12.3864
[50]	train-rmse:9.60728	test-rmse:10.3525
[100]	train-rmse:8.04975	test-rmse:9.38405
[150]	train-rmse:7.08566	test-rmse:8.9514
[200]	train-rmse:6.43059	test-rmse:8.73875
[250]	train-rmse:5.93347	test-rmse:8.59455
[300]	train-

[350]	train-rmse:4.00059	test-rmse:6.91897
[400]	train-rmse:3.78136	test-rmse:6.92152
376
[0]	train-rmse:11.0401	test-rmse:11.0472
[50]	train-rmse:8.55695	test-rmse:8.96432
[100]	train-rmse:7.24442	test-rmse:8.13966
[150]	train-rmse:6.47412	test-rmse:7.84283
[200]	train-rmse:5.9375	test-rmse:7.75422
[250]	train-rmse:5.54143	test-rmse:7.74363
237
[0]	train-rmse:11.4069	test-rmse:11.4147
[50]	train-rmse:8.8567	test-rmse:9.50304
[100]	train-rmse:7.41993	test-rmse:8.74737
[150]	train-rmse:6.5773	test-rmse:8.50754
[200]	train-rmse:5.92534	test-rmse:8.41087
195
[0]	train-rmse:12.35	test-rmse:12.3479
[50]	train-rmse:9.5997	test-rmse:10.1859
[100]	train-rmse:8.0442	test-rmse:9.27722
[150]	train-rmse:7.04959	test-rmse:8.90596
[200]	train-rmse:6.43024	test-rmse:8.62659
[250]	train-rmse:5.97659	test-rmse:8.57414
246
[0]	train-rmse:12.3905	test-rmse:12.3831
[50]	train-rmse:9.38761	test-rmse:9.95245
[100]	train-rmse:7.72381	test-rmse:9.03539
[150]	train-rmse:6.65759	test-rmse:8.73679
[200]	train-rm

In [25]:
xgb_level1_pred

[array([ 110.01844788,  103.94176483,  110.26132965,   94.75647736,
          98.13419342,  110.32420349,  110.24539185,   81.59608459,
          95.86064911,   82.7060318 ,   94.21020508,   93.49664307,
         108.13937378,   92.2779007 ,   94.0151062 ,  112.40760803,
          94.92800903,  110.00491333,   94.90766907,  110.24539185,
          93.91928101,   93.243927  ,   94.53507233,  112.02600861,
          93.78180695,  102.53238678,   96.09658813,   93.40581512,
         109.75544739,   94.52978516,  105.77883148,  109.63999939,
          94.0151062 ,  110.13648224,   97.90113831,   96.63822174,
         101.65377808,   81.60021973,  109.96286774,   94.45398712,
         112.08888245,   94.52978516,  103.74768829,   96.09658813,
          95.4442749 ,  109.67305756,  112.14144897,  101.93507385,
         112.14144897,  101.41082764,   93.50370789,   96.2204361 ,
          94.45606995,   93.59788513,   81.56393433,   94.11185455,
         101.43527222,   96.83412933,  110.26132